# Hello Researchers, in this notebook, we have performed various downstream tasks, such as viewing the maps of different upsampled GWL, analyzing feature importance, and examining SHAP values of the upsampled model

In [ ]:
import pandas as pd
import joblib

In [ ]:
!pip install -q openpyxl
!pip install -q xlrd


## Loading the low resolution data
### These are the GLDAS data and their vegetation indices.

In [ ]:
gldas_veg_index=pd.read_excel('/kaggle/input/new-data/GLDAS1_Gridwise_ndvi_ndwi_14Feb2025.xls')
gldas=pd.read_excel('/kaggle/input/new-data/GLDAS1_Gridwise_14HgfValue_flood_clay_and GWL 13Feb2025.xls')


In [ ]:
gldas = gldas.merge(gldas_veg_index, on='SerialID', how='inner').rename(columns={'SerialID':'GLDAS_SerialID','X':'POINT_X','Y':'POINT_Y'})


In [ ]:
low_res = pd.read_csv('/kaggle/input/test-gldas-data/gldas_data_yearwise.csv')

## Function to split the years

In [ ]:
import pandas as pd

def reshape_by_year(df):
    """
    Reshape the input dataframe so that each row corresponds to a specific year with 
    NDVI, NDWI, Min-GWL, and Max-GWL values for that year.
    
    Parameters:
    df (pd.DataFrame): Original dataframe containing the yearly data in separate columns.
    
    Returns:
    pd.DataFrame: Reshaped dataframe with one row per year containing NDVI, NDWI, Min-GWL, and Max-GWL.
    """
    # Step 1: Create a list of years
    years = list(range(2003, 2025))  # Assuming the years range from 2001 to 2023

    # Step 2: Prepare a dataframe for each year with NDWI, NDVI, and Min/Max GWL
    reshaped_df = pd.DataFrame()
#     print(df.shape)
    for year in years:
        # Filter the columns related to the current year
        year_columns = [f'NDVI{year}', f'NDWI{year}', f'Min-gwl_{year}', f'Max-gwl_{year}']
        
        # Create a temporary dataframe for the current year
        year_df = df[low_res.drop(columns=['Unnamed: 0','NDVI', 'NDWI', 'Min_GWS',
       'Max_GWS', 'Year']).columns.tolist()].copy()

        # Add the NDVI, NDWI, and GWL columns for the current year
        year_df['NDVI'] = df[f'ndvi{year}']
        year_df['NDWI'] = df[f'ndwi{year}']
        year_df['Min_GWS'] = df[f'Min_GWS_{year}']
        year_df['Max_GWS'] = df[f'Max_GWS_{year}']
        
        # Add the year as a column
        year_df['Year'] = year
#         print(reshaped_df.shape)
        # Append the temporary dataframe to the final dataframe
        reshaped_df = pd.concat([reshaped_df, year_df], ignore_index=True)

    # Step 3: Return the reshaped dataframe
    return reshaped_df




In [ ]:
new_low=reshape_by_year(gldas[low_res.drop(columns=['Unnamed: 0','NDVI', 'NDWI', 'Min_GWS',
       'Max_GWS', 'Year']).columns.tolist()+
gldas.columns[gldas.columns.str.startswith('nd')].tolist()
+ gldas.columns[gldas.columns.str.contains('GWS')].tolist()])

In [ ]:
new_low.columns

In [ ]:
# for i in gldas.columns:
#     print(i)

In [ ]:
import pandas as pd

def reshape_by_year(df):
    """
    Reshape the input dataframe so that each row corresponds to a specific year with 
    NDVI, NDWI, Min-GWL, and Max-GWL values for that year.
    
    Parameters:
    df (pd.DataFrame): Original dataframe containing the yearly data in separate columns.
    
    Returns:
    pd.DataFrame: Reshaped dataframe with one row per year containing NDVI, NDWI, Min-GWL, and Max-GWL.
    """
    # Step 1: Create a list of years
    years = list(range(2003, 2025))  # Assuming the years range from 2001 to 2023

    # Step 2: Prepare a dataframe for each year with NDWI, NDVI, and Min/Max GWL
    reshaped_df = pd.DataFrame()
#     print(df.shape)
    for year in years:
        # Filter the columns related to the current year
        year_columns = [f'NDVI{year}', f'NDWI{year}', f'Min-gwl_{year}', f'Max-gwl_{year}']
        
        # Create a temporary dataframe for the current year
        # year_df = df[[ 'POINT_X', 'POINT_Y', 
        #                'TWI', 'TRI', 'Sy', 'STI', 'SPI', 'Slope', 
        #               'Profile_curvature', 'Plan_curvature', 'lithology', 'lithology_clay_thickness', 
        #               'Distance_from_stream', 'elevation', 'drainage_density', 'Curvature', 
        #               'Aspect', 'GLDAS_SerialID']].copy()
        year_df = df
        # Add the NDVI, NDWI, and GWL columns for the current year
        year_df['NDVI'] = df[f'NDVI{year}']
        year_df['NDWI'] = df[f'NDWI{year}']
        # year_df['Min_GWL'] = df[f'Min-gwl_{year}']
        # year_df['Max_GWL'] = df[f'Max-gwl_{year}']
        
        # Add the year as a column
        year_df['Year'] = year
#         print(reshaped_df.shape)
        # Append the temporary dataframe to the final dataframe
        reshaped_df = pd.concat([reshaped_df, year_df], ignore_index=True)

    # Step 3: Return the reshaped dataframe
    return reshaped_df




## 2km uniform grid center points


In [ ]:
new_high_res=pd.read_excel("/kaggle/input/new-data/2k_points_14Hgfs_ndvi_ndwi_14Feb2025.xls")

## Points where people usually take measurements

#### In these location various measurements have been taken by various organizations

In [ ]:
import pandas as pd
original=pd.read_csv('/kaggle/input/new-data/new_insitu_points.csv',low_memory=False)


In [ ]:
original.columns

### GLDAS ID for the 2km uniform points

In [ ]:
high_gldas_id=pd.read_excel('/kaggle/input/new-data/2kpoints_gldas_grid 19Sept2024 (1).xls').rename(columns={'2k_Points_UniqID':'UniqID'})

In [ ]:
new_high_res=new_high_res.merge(high_gldas_id,on=['UniqID'])

In [ ]:
import re

new_high_res.columns = [re.sub(r'(ndvi|ndwi)(\d{2})$', lambda m: f"{m.group(1).upper()}20{m.group(2)}", col) for col in new_high_res.columns]



In [ ]:
new_high_res.columns

In [ ]:
imputer_col=['lithology_clay_thickness', 'TWI', 'TRI', 'Sy', 'STI', 'SPI', 'Slope', 
'Profile_curvature', 'Plan_curvature', 'Distance_from_stream', 'elevation', 
'drainage_density', 'Curvature', 'Aspect', 'lithology', 'NDVI2001', 'NDVI2002',
'NDVI2003', 'NDVI2004', 'NDVI2005', 'NDVI2006', 'NDVI2007', 'NDVI2008', 'NDVI2009',
'NDVI2010', 'NDVI2011', 'NDVI2012', 'NDVI2013', 'NDVI2014', 'NDVI2015', 'NDVI2016', 
'NDVI2017', 'NDVI2018', 'NDVI2019', 'NDVI2020', 'NDVI2021', 'NDVI2022', 'NDVI2023', 
'NDWI2001', 'NDWI2002', 'NDWI2003', 'NDWI2004', 'NDWI2005', 'NDWI2006', 'NDWI2007', 
'NDWI2008', 'NDWI2009', 'NDWI2010', 'NDWI2011', 'NDWI2012', 'NDWI2013', 'NDWI2014',
'NDWI2015', 'NDWI2016', 'NDWI2017', 'NDWI2018', 'NDWI2019', 'NDWI2020', 'NDWI2021', 
'NDWI2022', 'NDWI2023']

### Imputing the missing values

In [ ]:
imputer=joblib.load('/kaggle/input/test-wights-for-groundhog/imputer.pkl')
new_high_res[imputer_col]=imputer.transform(new_high_res[imputer_col])

In [ ]:
imputer=joblib.load('/kaggle/input/test-wights-for-groundhog/imputer.pkl')
original[imputer_col]=imputer.transform(original[imputer_col])

In [ ]:
new_high_res.columns

## Reshaping for easier use case

In [ ]:
new_high=reshape_by_year(new_high_res)

In [ ]:
original=reshape_by_year(original)

In [ ]:
new_high.isnull().any()

## Trigger Download

In [ ]:
from IPython.display import FileLink, display, HTML, Javascript

def trigger_download(filename, my_id=0):
    # Create the FileLink object
    file_link = FileLink(filename)

    # Get the path of the file
    file_path = file_link.path

    # Create the HTML link
    html = f'<a id="download_link_{file_path}_{my_id}" href="{file_path}" download>{file_path}</a>'

    # Display the HTML link
    display(HTML(html))

    # Create and run the JavaScript to automatically click the link
    js_code = f'''
    var link = document.getElementById('download_link_{file_path}_{my_id}');
    link.click();
    '''
    display(Javascript(js_code))




## These data only have upto 2022

In [ ]:
import joblib
import pandas as pd
import numpy as np
import torch

# Load datasets
high_res = pd.read_csv('/kaggle/input/test-gldas-data/resolution_2k.csv').drop(columns=['Unnamed: 0'])
low_res = pd.read_csv('/kaggle/input/test-gldas-data/gldas_data_yearwise.csv')

In [ ]:
# # low_res.columns=low_res.columns.str.replace('_',' ')
# # Replace underscores with spaces and add "Representative" prefix
# low_res.columns = 'Representative ' + low_res.columns.str.replace('_', ' ')

# low_res.columns

In [ ]:
joblib.load('/kaggle/input/test-wights-for-groundhog/standard_scaler.pkl').feature_names_in_

# Downstream task with upsampling model
## You can choose which points you want to test
- ### you can test 2 km uniform grid center points
- ### or you can test the points where in-situ measurements are usually taken

In [ ]:
import joblib
import pandas as pd
import numpy as np
import torch

# Load datasets
# high_res = pd.read_csv('/kaggle/input/test-gldas-data/resolution_2k.csv')
# low_res = pd.read_csv('/kaggle/input/test-gldas-data/gldas_data_yearwise.csv')
choice = input("Select for which scenario you want to see the results for:\n (1 for uniform 2km resolution grid points, 2 for in-situ measuring points): ")

if choice == "1":
    high_res = new_high
elif choice == "2":
    high_res = original
else:
    print("Invalid choice. Defaulting to uniform 2km resolution grid points.")
    high_res = new_high

low_res=new_low
# Load scalers and model
fitted_scaler = joblib.load('/kaggle/input/test-wights-for-groundhog/fitted_scaler.pkl')
standard_scaler = joblib.load('/kaggle/input/test-wights-for-groundhog/standard_scaler.pkl')
rf_model = joblib.load('/kaggle/input/test-wights-for-groundhog/upscaling_model.joblib')

# standard_scaler = joblib.load('/kaggle/input/weights-for-upsampling-2/scaler_for_high_low.joblib')
# rf_model = joblib.load('/kaggle/input/weights-for-upsampling-2/upscaling_model.joblib')

# standard_scaler = joblib.load('/kaggle/input/updated-upsampling-weights/scaler_for_high_low.joblib')
# rf_model = joblib.load('/kaggle/input/updated-upsampling-weights/upscaling_model (1).joblib')

high_res['Original_Sy']=high_res.Sy
# Transform high-resolution data
high_res[fitted_scaler.feature_names_in_] = fitted_scaler.transform(high_res[fitted_scaler.feature_names_in_])

# Merge datasets
data = high_res.merge(low_res, on=['GLDAS_SerialID', 'Year']).rename(columns={'POINT_X_x': 'POINT_X', 'POINT_Y_x': 'POINT_Y'})

# Scale numerical features
X_numerical_scaled = standard_scaler.transform(data[standard_scaler.feature_names_in_])
# print(data.lithology.unique(),high_res.lithology.unique())
# Encode categorical data
X_cat = data[['lithology', 'lithology_MAJORITY']].astype('string')
X_cat_encoded = pd.get_dummies(X_cat)
# print(X_cat_encoded.columns)

# print(X_numerical_scaled.shape, X_cat_encoded.shape)
# Prepare final input tensor
X_final = torch.tensor(np.hstack([X_numerical_scaled, X_cat_encoded]).astype('float32'))
print(X_numerical_scaled.shape, X_cat_encoded.shape)
# Predict
pred = rf_model.predict(X_final)
high_res['Sy'] = high_res['Sy'] if 'Sy' in high_res.columns else 1  # Ensure Sy column exists
recharge = (pred[:, 1] - pred[:, 0]) * 100 * data['Original_Sy']

# Display feature names and feature importance
feature_names = list(standard_scaler.feature_names_in_) + list(X_cat_encoded.columns)
feature_importance = rf_model.feature_importances_

# Create DataFrame for easier viewing
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Display feature importance
print(importance_df)


In [ ]:
# 'lithology_1', 'lithology_10', 'lithology_11', 'lithology_12',
#        'lithology_13', 'lithology_14', 'lithology_15', 'lithology_16',
#        'lithology_17', 'lithology_18', 'lithology_19', 'lithology_2',
#        'lithology_20', 'lithology_21', 'lithology_22', 'lithology_23',
#        'lithology_24', 'lithology_25', 'lithology_26', 'lithology_27',
#        'lithology_3', 'lithology_4', 'lithology_5', 'lithology_6',
#        'lithology_7', 'lithology_8', 'lithology_9', 'lithology_MAJORITY_1',
#        'lithology_MAJORITY_10', 'lithology_MAJORITY_11',
#        'lithology_MAJORITY_12', 'lithology_MAJORITY_14',
#        'lithology_MAJORITY_15', 'lithology_MAJORITY_16',
#        'lithology_MAJORITY_17', 'lithology_MAJORITY_18',
#        'lithology_MAJORITY_19', 'lithology_MAJORITY_2',
#        'lithology_MAJORITY_20', 'lithology_MAJORITY_22',
#        'lithology_MAJORITY_23', 'lithology_MAJORITY_24',
#        'lithology_MAJORITY_26', 'lithology_MAJORITY_27',
#        'lithology_MAJORITY_3', 'lithology_MAJORITY_4', 'lithology_MAJORITY_5',
#        'lithology_MAJORITY_6', 'lithology_MAJORITY_7', 'lithology_MAJORITY_8',
#        'lithology_MAJORITY_9'],

In [ ]:
feature_names=['TWI', 'TRI', 'Sy', 'STI', 'SPI', 'Slope', 'Profile curvature',
       'Plan curvature', 'lithology clay thickness',
       'Distance from stream', 'elevation', 'drainage density',
       'Curvature', 'Aspect', 'mean NDVI', 'mean NDWI', 'Representative Curvature',
       'Representative Slope','Representative Profile curvature', 'Representative Plan curvature',
       'Representative Distance from stream', 'Representative Aspect', 'Representative drainage density',
       'Representative Elevation', 'Representative SPI', 'Representative STI',
       'Representative Sy', 'Representative TRI', 'Representative TWI',
       'Representative lithology clay thickness', 'Representative NDVI', 'Representative NDWI', 'Min GWS (GLDAS)',
       'Max GWS (GLDAS)']+ list(X_cat_encoded.columns)

In [ ]:
print(len(['TWI', 'TRI', 'Sy', 'STI', 'SPI', 'Slope', 'Profile curvature',
       'Plan curvature', 'lithology clay thickness',
       'Distance from stream', 'elevation', 'drainage density',
       'Curvature', 'Aspect', 'mean NDVI', 'mean NDWI', 'Representative Curvature',
       'Representative Slope','Representative Profile curvature', 'Representative Plan curvature',
       'Representative Distance from stream (MAJORITY)', 'Representative Aspect (MAJORITY) ', 'Representative drainage density MAJORITY',
       'Representative Elevation (MAJORITY)', 'Representative SPI (MAJORITY)', 'Representative STI (MAJORITY)',
       'Representative Sy (MAJORITY)', 'Representative TRI (MAJORITY)', 'Representative TWI (MAJORITY)',
       'Representative lithology clay thickness (MAJORITY)', 'Representative NDVI (MAJORITY)', 'Representative NDWI (MAJORITY)', 'Min GWS (GLDAS)',
       'Max GWS (GLDAS)']))

In [ ]:
from sklearn.model_selection import train_test_split
_,X_test=train_test_split(X_final,test_size=0.2)

# SHAP for the upsampling model

In [ ]:
import shap
import pandas as pd
import matplotlib.pyplot as plt

# Initialize SHAP TreeExplainer with approximate mode
explainer = shap.TreeExplainer(rf_model, approximate=True, feature_names=feature_names)

# Compute SHAP values for X_test
shap_values = explainer.shap_values(X_test.numpy(), approximate=True)


In [ ]:
np.array(shap_values).shape,X_test.shape

In [ ]:
import matplotlib.pyplot as plt
import shap
# Set font size globally using matplotlib
plt.rcParams.update({
    'font.size': 20,      # General font size
    'axes.titlesize': 20, # Title font size
    'axes.labelsize': 20, # Axis label font size
    'xtick.labelsize': 20, # X-tick label size
    'ytick.labelsize': 16, # Y-tick label size
})
# Create a bar plot
plt.figure(figsize=(10, 10))
# Generate the summary plot for the first 16 features
shap.summary_plot(
    shap_values[1][:, :34],  # Only the first 16 features
    X_test.numpy()[:, :34],  # Only the first 16 features
    feature_names=feature_names[:34],  # Names of the first 16 features
    # matplotlib=True,
     max_display=34,  # Show all 34 features
    show=False  # Prevent the plot from being displayed
)
# Customize the y-axis tick labels to align them to the left
# plt.gca().set_yticklabels(plt.gca().get_yticklabels(), ha='left')

# # Adjust the layout to ensure labels fit well
# plt.gcf().subplots_adjust(left=0.9)  # Increase left margin to avoid truncation


# Save the figure
plt.savefig("upsample_max_shap_summary_plot.png", dpi=300, bbox_inches='tight')  # Save as PNG with high resolution
plt.close()  # Close the plot to avoid display

trigger_download('upsample_max_shap_summary_plot.png')


## Assign those Values

In [ ]:
data['Max GWL']=pred[:,1]
data['Min GWL']=pred[:,0]
data['recharge']=recharge

In [ ]:
data[['District', 'Upazila']]=original[['District', 'Upazila']]

In [ ]:
data

In [ ]:
data[data.POINT_X<90].sort_values(by=['Max GWL'], ascending=False)[['POINT_X', 'POINT_Y','District','Max GWL','Year' ]].head(10).values.tolist()


In [ ]:
import pandas as pd

# Ensure data is sorted properly
data = data.sort_values(by=["POINT_X", "POINT_Y", "Year"])

# Get first and last year's values for each location
first_values = data.groupby(["POINT_X", "POINT_Y"]).first().reset_index()
last_values = data.groupby(["POINT_X", "POINT_Y"]).last().reset_index()

# Compute absolute change for Max GWL
first_values["Max_GWL_Change"] = last_values["Max GWL"] - first_values["Max GWL"]

# Store the Max GWL values for first and last year
first_values["First_Year_Max_GWL"] = first_values["Max GWL"]
first_values["Last_Year_Max_GWL"] = last_values["Max GWL"]

# Keep relevant columns: Max_GWL_Change, District, First_Year_Max_GWL, and Last_Year_Max_GWL
result = first_values[["POINT_X", "POINT_Y", "District", "Max_GWL_Change", "First_Year_Max_GWL", "Last_Year_Max_GWL"]]

# Get overall statistics
max_change = result["Max_GWL_Change"].max()
min_change = result["Max_GWL_Change"].min()
avg_change = result["Max_GWL_Change"].mean()
std_change = result["Max_GWL_Change"].std()

# Find the locations with max and min change
max_change_location = result[result["Max_GWL_Change"] == max_change]
min_change_location = result[result["Max_GWL_Change"] == min_change]

# Print results
print(f"Maximum Change in Max GWL: {max_change:.2f} at:")
print(max_change_location)

print(f"\nMinimum Change in Max GWL: {min_change:.2f} at:")
print(min_change_location)

print(f"\nAverage Change in Max GWL: {avg_change:.2f}")
print(f"Standard Deviation in Max GWL: {std_change:.2f}")

# Aggregate by district to find where changes are highest
district_changes = result.groupby("District")["Max_GWL_Change"].mean().sort_values(ascending=False)

print("\nTop Districts with the Most Change in Max GWL:")
print(district_changes.head(10))  # Show top 10 districts


In [ ]:
from scipy.stats import theilslopes
# Sample dataframe
df = data  # Replace with actual data

# Group data by POINT_X and POINT_Y and calculate Sen's Slope in one step
def calculate_sens_slope(group):
    if group['Year'].nunique() > 1:
        slope, intercept, lower, upper = theilslopes(group['recharge'], group['Year'])
        return round(slope, 2)  # Round slope to 2 decimal places
    return np.nan  # Not enough data

# Calculate the Sen's Slope for each group and reset index
trends_df = df.groupby(['POINT_X', 'POINT_Y']).apply(calculate_sens_slope).reset_index(name='sens_slope')

# Drop rows with NaN values in sens_slope
trends_df = trends_df.dropna(subset=['sens_slope'])

In [ ]:
trends_df[(trends_df.POINT_X==90.4167)&(trends_df.POINT_Y==23.7458)]

In [ ]:
data[(data.POINT_X==90.4167)&(data.POINT_Y==23.7458)&((data.Year==2003)|(data.Year==2024))]

In [ ]:
data[['Year']].head(3)

In [ ]:
import pandas as pd

# Ensure data is sorted properly
data = data.sort_values(by=["POINT_X", "POINT_Y", "Year"])

# Get first and last year's values for each location
first_values = data.groupby(["POINT_X", "POINT_Y"]).first().reset_index()
last_values = data.groupby(["POINT_X", "POINT_Y"]).last().reset_index()

# Compute absolute change
first_values["recharge_Change"] = last_values["recharge"] - first_values["recharge"]

# Keep relevant columns: Max_GWL_Change and District
result = first_values[["POINT_X", "POINT_Y", "District", "recharge_Change"]]

# Get overall statistics
max_change = result["recharge_Change"][result.District=="Dhaka"].max()
min_change = result["recharge_Change"][result.District=="Dhaka"].min()
avg_change = result["recharge_Change"].mean()
std_change = result["recharge_Change"].std()

# Find the locations with max and min change
max_change_location = result[result["recharge_Change"] == max_change]
min_change_location = result[result["recharge_Change"] == min_change]

# Print results
print(f"Maximum Change in recharge: {max_change:.2f} at:")
print(max_change_location)

print(f"\nMinimum Change in recharge : {min_change:.2f} at:")
print(min_change_location)

print(f"\nAverage Change in recharge: {avg_change:.2f}")
print(f"Standard Deviation in recharge: {std_change:.2f}")

# Aggregate by district to find where changes are highest
district_changes = result.groupby("District")["recharge_Change"].mean().sort_values(ascending=False)

print("\nTop Districts with the Most Change in recharge:")
print(district_changes.head(10))  # Show top 10 districts


In [ ]:
import pandas as pd

# Ensure data is sorted properly
data = data.sort_values(by=["POINT_X", "POINT_Y", "Year"])

# Get first and last year's values for each location
first_values = data.groupby(["POINT_X", "POINT_Y"]).first().reset_index()
last_values = data.groupby(["POINT_X", "POINT_Y"]).last().reset_index()

# Compute absolute change for Min GWL
first_values["Min_GWL_Change"] = last_values["Min GWL"] - first_values["Min GWL"]

# Keep relevant columns: Min_GWL_Change and District
result = first_values[["POINT_X", "POINT_Y", "District", "Min_GWL_Change"]]

# Get overall statistics
max_change = result["Min_GWL_Change"].max()
min_change = result["Min_GWL_Change"].min()
avg_change = result["Min_GWL_Change"].mean()
std_change = result["Min_GWL_Change"].std()

# Find the locations with max and min change
max_change_location = result[result["Min_GWL_Change"] == max_change]
min_change_location = result[result["Min_GWL_Change"] == min_change]

# Print results
print(f"Maximum Change in Min GWL: {max_change:.2f} at:")
print(max_change_location)

print(f"\nMinimum Change in Min GWL: {min_change:.2f} at:")
print(min_change_location)

print(f"\nAverage Change in Min GWL: {avg_change:.2f}")
print(f"Standard Deviation in Min GWL: {std_change:.2f}")

# Aggregate by district to find where changes are highest
district_changes = result.groupby("District")["Min_GWL_Change"].mean().sort_values(ascending=False)

print("\nTop Districts with the Most Change in Min GWL:")
print(district_changes.head(10))  # Show top 10 districts


In [ ]:
# pip install --upgrade geopy

In [ ]:
# from geopy.geocoders import Nominatim
# import time

# # List of coordinates in [longitude, latitude] format.
# coords = [
#     [90.6064, 23.9494],
#     [90.5486, 23.9619],
#     [90.4614, 23.8314],
#     [90.5989, 23.7844],
#     [90.4167, 23.7458],
#     [90.391776, 23.759484]
# ]

# # Initialize Nominatim with a custom user_agent.
# geolocator = Nominatim(user_agent="geoapiExercises", timeout=10)

# def reverse_geocode(lon, lat, retries=3):
#     for attempt in range(retries):
#         try:
#             # Note: geopy expects coordinates as (latitude, longitude)
#             location = geolocator.reverse((lat, lon), exactly_one=True)
#             if location and location.address:
#                 return location.address
#             else:
#                 return "Address not found"
#         except Exception as e:
#             print(f"Attempt {attempt + 1} failed with error: {e}")
#             time.sleep(2)  # wait before retrying
#     return "Failed to retrieve address after retries"

# # Loop through coordinates and print the result.
# for pair in coords:
#     lon, lat = pair  # Original order: [longitude, latitude]
#     address = reverse_geocode(lon, lat)
#     print(f"Coordinates (lat, lon): ({lat}, {lon})")
#     print("Address:", address)
#     print("-" * 50)
#     time.sleep(1.5)  # extra delay to respect rate limits


In [ ]:
data.to_csv('predicted_data.csv')

In [ ]:
trigger_download('predicted_data.csv')

# Visualiztion

## Function for plotting

### The bar plot is only applicable if you have choosen the uniform 2km points. As we assumed each point is of 2km x 2km. So, a total of 4 km area in each point

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm, LinearSegmentedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

def plot_column_as_points(data, column_values, column_name, point_size=5):
    fig, ax = plt.subplots(1, 1, figsize=(20, 20))

    # Extract values for the current column
    longitudes = data.POINT_X
    latitudes = data.POINT_Y

    # Define your ranges and corresponding colors
    boundaries = [0, 5.3, 7.6, 9.8, 11.3, 15, 20.5, 26, 35.5, 58, 68, 78]
    cmap_colors = [
        'blue',         # below 0
        '#a8e1e0',      # 0 to < 5.3 (light blue)
        '#66c18a',      # 5.3 to < 7.6 (light green)
        '#3b7a3d',      # 7.6 to < 9.8 (dark green)
        '#f3d5a4',      # 9.8 to < 11.3 (light purple)
        '#b299ca',      # 11.3 to < 15 (purple)
        '#e4a6a3',      # 15 to < 20.5
        '#d35d60',      # 20.5 to < 26 (red)
        '#a0322e',      # 26 to < 35.5 (dark red)
        '#88322e',      # 35.5 to < 58
        '#55322e',      # 58 to < 68
        '#330e0f',      # 68 to < 78 (dark gray)
    ]

    # Create colormap
    cmap = LinearSegmentedColormap.from_list("custom_cmap", cmap_colors)
    norm = BoundaryNorm(boundaries, cmap.N)

    # Scatter plot
    sc = ax.scatter(longitudes, latitudes, c=column_values, cmap=cmap, norm=norm, s=30, edgecolor='None', alpha=1.0)
    cbar = plt.colorbar(sc, ax=ax)
    cbar.set_label(f'{column_name}', fontsize=22.5)  # Increased fontsize
    cbar.ax.tick_params(labelsize=19)  # Increase colorbar tick size

    ax.set_xlabel('Longitude', fontsize=22.5)
    ax.set_ylabel('Latitude', fontsize=22.5)
    ax.set_title(f'{column_name} ',fontsize=27)
    ax.tick_params(axis='both', labelsize=20)  # Increase tick labels size

    # Bin the data
    labels = [f"{boundaries[i]}-{boundaries[i+1]}" for i in range(len(boundaries) - 1)]
    data['Range'] = pd.cut(column_values, bins=boundaries, labels=labels, right=False)

    # Count occurrences and calculate area
    range_counts = data['Range'].value_counts().reindex(labels, fill_value=0)
    areas = (range_counts * 4) / 1000  # Convert km² to thousand km²
    total_area = areas.sum()

    # Create inset bar plot inside scatter plot (top-right corner)
    ax_inset = inset_axes(ax, width="45%", height="23%", loc='upper right')

    bars = ax_inset.bar(range_counts.index, areas, color=cmap_colors[:len(range_counts) + 1])
    ax_inset.set_xticklabels(range_counts.index, rotation=45, ha='right', fontsize=12)  # Increased fontsize
    ax_inset.set_ylabel('Area (1000 km²)', fontsize=15)

    # Add area values and percentages on top of bars
    for bar, area in zip(bars, areas):
        height = bar.get_height()
        if height > 0:
            percentage = (area / total_area) * 100
            ax_inset.text(bar.get_x() + bar.get_width() / 2, height + 0.2, 
                          f"{area:.2f}\n({percentage:.1f}%)", ha='center', va='bottom', 
                          fontsize=12, fontweight='bold')  # Increased fontsize

    plt.tight_layout()
    plt.savefig(f'{column_name}_scatter_inset_bar.png', dpi=300)
    plt.show()


## Visualizing Maximum and Minimum GWL (BGL) in meters

In [ ]:
cur_data=data
# cur_data=psuedo_data
for year in cur_data.Year.unique():
#     if year<=2001:
#         continue
    
    for mode in ['Min GWL','Max GWL']:
        A=cur_data[cur_data.Year==year].sort_values(by=['POINT_X','POINT_Y'])
        B=cur_data[cur_data.Year==year-1].sort_values(by=['POINT_X','POINT_Y'])
        x=A[mode].values.reshape(-1)
#         y=B['min_gwl'].values.reshape(-1)
        print(A.shape,x.shape)
        plot_column_as_points(A,x,f'Upsample {mode} (BGL) for the year {year} (meters)')

## Function for plotting GLDAS

### The bar plot is only applicable if you have choosen the uniform 2km points. As we assumed each point is of 2km x 2km. So, a total of 4 km area in each point

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm, LinearSegmentedColormap

def plot_column_as_points(data, column_values, column_name,point_size=5):
    fig, ax = plt.subplots(1, 1, figsize=(20, 20))

    # Extract values for the current column
    longitudes = data.POINT_X
    latitudes = data.POINT_Y

    # Define your ranges and corresponding colors
    boundaries = [0, 500, 700, 900, 1100, 1300, 1500, 1700, 1900,2000 ]
    cmap_colors = [
        'blue',         # below 0
        '#a8e1e0',      # < 5.3 (light blue)
        '#66c18a',      # 5.3 to < 7.6 (light green)
        '#3b7a3d',      # 7.6 to < 9.8 (dark green)
        '#f3d5a4',      # 9.8 to < 11.3 (light purple)
        '#b299ca',      # 11.3 to < 15 (purple)
        '#e4a6a3',      # 15 to < 20.5 (-0.5 to 0d)
        '#d35d60',      # 20.5 to < 26 (red)
        '#a0322e',      # 26 to < 35.5 (dark red)
        '#330e0f',      # 35.5 to < 58 (dark gray)
    ]

    # Create a custom colormap using LinearSegmentedColormap
    cmap = LinearSegmentedColormap.from_list("custom_cmap", cmap_colors)
    norm = BoundaryNorm(boundaries, cmap.N)

    # # Scatter plot with discrete color mapping
    # sc = ax.scatter(longitudes, latitudes, c=column_values, cmap=cmap, norm=norm, s=point_size, edgecolor='None')

    # # Add a color bar
    # cbar = plt.colorbar(sc, ax=ax)
    # cbar.set_label(f'{column_name}')

    # Scatter plot
    sc = ax.scatter(longitudes, latitudes, c=column_values, cmap=cmap, norm=norm, s=30, edgecolor='None', alpha=1.0)
    cbar = plt.colorbar(sc, ax=ax)
    cbar.set_label(f'{column_name}', fontsize=22.5)  # Increased fontsize
    cbar.ax.tick_params(labelsize=19)  # Increase colorbar tick size

    ax.set_xlabel('Longitude', fontsize=22.5)
    ax.set_ylabel('Latitude', fontsize=22.5)
    ax.set_title(f'{column_name} ',fontsize=27)
    ax.tick_params(axis='both', labelsize=20)  # Increase tick labels size
    # Bin the data
    labels = [f"{boundaries[i]}-{boundaries[i+1]}" for i in range(len(boundaries) - 1)]
    data['Range'] = pd.cut(column_values, bins=boundaries, labels=labels, right=False)

    # Count occurrences and calculate area
    range_counts = data['Range'].value_counts().reindex(labels, fill_value=0)
    areas = (range_counts * 4) / 1000  # Convert km² to thousand km²
    total_area = areas.sum()

    # Create inset bar plot inside scatter plot (top-right corner)
    ax_inset = inset_axes(ax, width="45%", height="23%", loc='upper right')

    bars = ax_inset.bar(range_counts.index, areas, color=cmap_colors[:len(range_counts) + 1])
    ax_inset.set_xticklabels(range_counts.index, rotation=45, ha='right', fontsize=12)  # Increased fontsize
    ax_inset.set_ylabel('Area (1000 km²)', fontsize=15)

    # Add area values and percentages on top of bars
    for bar, area in zip(bars, areas):
        height = bar.get_height()
        if height > 0:
            percentage = (area / total_area) * 100
            ax_inset.text(bar.get_x() + bar.get_width() / 2, height + 0.2, 
                          f"{area:.2f}\n({percentage:.1f}%)", ha='center', va='bottom', 
                          fontsize=12, fontweight='bold')  # Increased fontsize

    # Save the plot to file
    plt.savefig(f'{column_name}.png', dpi=300)
    print(column_name)
    plt.show()

# Example Usage
# Assuming data is a DataFrame containing column_name values and 'longitudes', 'latitudes' as separate arrays
# plot_column_as_points(data, column_values, 'GWL_Column')


In [ ]:
low_res.Max_GWS.max()

In [ ]:
cur_data=data
# cur_data=psuedo_data
for year in cur_data.Year.unique():
#     if year<=2001:
#         continue
    
    for mode in ['Min_GWS','Max_GWS']:
        A=cur_data[cur_data.Year==year].sort_values(by=['POINT_X','POINT_Y'])
        B=cur_data[cur_data.Year==year-1].sort_values(by=['POINT_X','POINT_Y'])
        x=A[mode].values.reshape(-1)
#         y=B['min_gwl'].values.reshape(-1)
        print(A.shape,x.shape)
        plot_column_as_points(A,x,f'GLDAS low res {mode.replace("_"," ")}  for the year {year} (mm)')

## Function for plotting Recharge

### The bar plot is only applicable if you have choosen the uniform 2km points. As we assumed each point is of 2km x 2km. So, a total of 4 km area in each point

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm, LinearSegmentedColormap

def plot_column_as_points(data,column_values,column_name):
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))

    # Extract values for the current column
    longitudes=data.POINT_X
    latitudes=data.POINT_Y

    # Define your ranges and corresponding colors
    boundaries = [0, 5.3, 7.6, 9.8, 11.3, 15, 20.5, 26, 35.5, 58, 60, 70, 80, 90, 100, 150]
    cmap_colors = [
        'blue',         # below 0
        '#a8e1e0',      # < 5.3 (light blue)
        '#66c18a',      # 5.3 to < 7.6 (light green)
        '#3b7a3d',      # 7.6 to < 9.8 (dark green)
        '#f3d5a4',      # 9.8 to < 11.3 (light purple)
        '#b299ca',      # 11.3 to < 15 (purple)
        '#e4a6a3',      # 15 to < 20.5 (-0.5 to 0d)
        '#d35d60',      # 20.5 to < 26 (red)
        '#a0322e',      # 26 to < 35.5 (dark red)
        '#330e0f',      # 35.5 to < 58 (dark gray)
        '#4f4d4d',      # 58 to < 60 (gray)
        '#7d7b7b',      # 60 to < 70 (light gray)
        '#a9a8a8',      # 70 to < 80 (lighter gray)
        '#c2c0c0',      # 80 to < 90 (lightest gray)
        '#dbdbdb',      # 90 to < 100 (almost white)
        'black'         # 100+ (black)
    ]

    # Create a custom colormap using LinearSegmentedColormap
    cmap = LinearSegmentedColormap.from_list("custom_cmap", cmap_colors)
    norm = BoundaryNorm(boundaries, cmap.N)

    # Scatter plot with discrete color mapping
    sc = ax.scatter(longitudes, latitudes, c=column_values, cmap=cmap, norm=norm, s=5, edgecolor='None')

    # Add a color bar
    cbar = plt.colorbar(sc, ax=ax)
    cbar.set_label(f'{column_name} ')

    # Add title and labels
    ax.set_title(f'{column_name} GWL Points ', fontsize=16)
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')

    # Save the plot to file
    plt.savefig(f'{column_name}.png', dpi=300)
    print(column_name)
    plt.show()

# Example Usage
# Assuming data is a DataFrame containing column_name values and 'longitudes', 'latitudes' as separate arrays
# plot_column_as_points('GWL_Column', data, longitudes, latitudes)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm, LinearSegmentedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

def plot_column_with_bar(data, column_values, column_name):
    fig, ax = plt.subplots(figsize=(20, 20))  # Main scatter plot

    # Extract values for the current column
    longitudes = data.POINT_X
    latitudes = data.POINT_Y

    # Define ranges and colors
    boundaries = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 70, 80, 100]
    cmap_colors = [
        'blue', '#a8e1e0', '#66c18a', '#f4a700',  # Changed deep green to deep yellow
        '#f3d5a4', '#b299ca', '#e4a6a3', '#d35d60',
        '#a0322e', '#330e0f', '#4f4d4d', '#7d7b7b',
        '#a9a8a8', '#c2c0c0', '#dbdbdb', 'black'
    ]

    # Create colormap
    cmap = LinearSegmentedColormap.from_list("custom_cmap", cmap_colors)
    norm = BoundaryNorm(boundaries, cmap.N)

    # Scatter plot
    sc = ax.scatter(longitudes, latitudes, c=column_values, cmap=cmap, norm=norm, s=30, edgecolor='None', alpha=1.0)
    cbar = plt.colorbar(sc, ax=ax)
    cbar.set_label(f'{column_name}', fontsize=22.5)  # Increased fontsize
    cbar.ax.tick_params(labelsize=19)  # Increase colorbar tick size

    ax.set_xlabel('Longitude', fontsize=22.5)
    ax.set_ylabel('Latitude', fontsize=22.5)
    ax.tick_params(axis='both', labelsize=20)  # Increase tick labels size
    # Bin the data
    labels = [f"{boundaries[i]}-{boundaries[i+1]}" for i in range(len(boundaries) - 1)]
    data['Range'] = pd.cut(column_values, bins=boundaries, labels=labels, right=False)

    # Count occurrences and calculate area
    range_counts = data['Range'].value_counts().reindex(labels, fill_value=0)
    areas = (range_counts * 4) / 1000  # Convert km² to thousand km²
    total_area = areas.sum()

    # Create inset bar plot inside scatter plot (top-right corner)
    ax_inset = inset_axes(ax, width="45%", height="23%", loc='upper right')

    bars = ax_inset.bar(range_counts.index, areas, color=cmap_colors[:len(range_counts) + 1])
    ax_inset.set_xticklabels(range_counts.index, rotation=45, ha='right', fontsize=12)  # Increased fontsize
    ax_inset.set_ylabel('Area (1000 km²)', fontsize=15)

    # Add area values and percentages on top of bars
    for bar, area in zip(bars, areas):
        height = bar.get_height()
        if height > 0:
            percentage = (area / total_area) * 100
            ax_inset.text(bar.get_x() + bar.get_width() / 2, height + 0.2, 
                          f"{area:.2f}\n({percentage:.1f}%)", ha='center', va='bottom', 
                          fontsize=9, fontweight='bold')  # Increased fontsize

    plt.tight_layout()
    plt.savefig(f'{column_name}_scatter_inset_bar.png', dpi=300)
    plt.show()

# Example Usage:
# plot_column_with_bar(data, data['GWL_Column'], 'GWL_Column')


## Visulaizing The Recharge

In [ ]:
len(cur_data[cur_data.Year==year])*4

In [ ]:
cur_data=data
# cur_data=psuedo_data
for year in cur_data.Year.unique():

    A=cur_data[cur_data.Year==year].sort_values(by=['POINT_X','POINT_Y'])
    B=cur_data[cur_data.Year==year-1].sort_values(by=['POINT_X','POINT_Y'])
    x=A['recharge'].values.reshape(-1)
#         y=B['min_gwl'].values.reshape(-1)
    print(A.shape,x.shape)
    plot_column_with_bar(A,x,f'Upsample Recharge for the year {year} (centimeters)')
    # break

## Zip 

In [ ]:
!zip GLDAS_GWS.zip *GWS*


In [ ]:
trigger_download('GLDAS_GWS.zip')

In [ ]:
!zip Upsampled_GWL.zip *GWL*


In [ ]:
!zip Upsampled_Recharge.zip *Recharge*

In [ ]:
trigger_download('Upsampled_GWL.zip')


In [ ]:
trigger_download('Upsampled_Recharge.zip')

# Visualizing the Trend 

### The bar plot is only applicable if you have choosen the uniform 2km points. As we assumed each point is of 2km x 2km. So, a total of 4 km area in each point

## Visualizing the Trend of Recharge

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import theilslopes
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# Sample dataframe
df = data  # Replace with actual data

# Group data by POINT_X and POINT_Y and calculate Sen's Slope in one step
def calculate_sens_slope(group):
    if group['Year'].nunique() > 1:
        slope, intercept, lower, upper = theilslopes(group['recharge'], group['Year'])
        return round(slope, 2)  # Round slope to 2 decimal places
    return np.nan  # Not enough data

# Calculate the Sen's Slope for each group and reset index
trends_df = df.groupby(['POINT_X', 'POINT_Y']).apply(calculate_sens_slope).reset_index(name='sens_slope')

# Drop rows with NaN values in sens_slope
trends_df = trends_df.dropna(subset=['sens_slope'])

# Define bins and labels for categorizing slope values
bins = [-1, -0.5, -0.3, -0.05, 0.05, 0.5, 1]
labels = ['-1 to -0.5', '-0.5 to -0.3', '-0.3 to -0.05', '-0.05 to 0.05', '0.05 to 0.5', '0.5 to 1']
trends_df['slope_category'] = pd.cut(trends_df['sens_slope'], bins=bins, labels=labels, right=False)

# Define the custom color palette for discrete categories
custom_palette = {
    '-1 to -0.5': '#000000',
    '-0.5 to -0.3': '#BB0000',
    '-0.3 to -0.05': '#FFA07A',
    '-0.05 to 0.05': "#FFFFFF",
    '0.05 to 0.5': '#ADD8E6',
    '0.5 to 1': '#00008B'
}

# Create figure and scatter plot
fig, ax = plt.subplots(figsize=(10, 12))

scatter = sns.scatterplot(
    data=trends_df,
    x='POINT_X',
    y='POINT_Y',
    hue='slope_category',
    palette=custom_palette,
    edgecolor=None,
    ax=ax
)

# Add title and labels
ax.set_title("Recharge Trend (Sen's Slope cm/year) for Bangladesh (2003 to 2022)", fontsize=16)
ax.set_xlabel('POINT_X')
ax.set_ylabel('POINT_Y')
ax.legend(title="Slope Category", loc='upper right')

# Count occurrences for the bar plot
category_counts = trends_df['slope_category'].value_counts().reindex(labels, fill_value=0)

# Convert counts to area (each point = 4 km²)
total_points = category_counts.sum()
category_area = (category_counts * 4) / 1000  # Convert to thousand km²
category_percent = (category_counts / total_points) * 100  # Calculate percentage

# Create inset bar plot
ax_inset = inset_axes(ax, width="50%", height="20%", loc='upper right')

bars = ax_inset.bar(
    category_counts.index, 
    category_area,  # Use area (in thousand km²) instead of count
    color=[custom_palette[label] for label in category_counts.index], 
    edgecolor='black',  # Black edges
    linewidth=0.5
)

ax_inset.set_xticklabels(category_counts.index, 
                         # rotation=45,
                         ha='center', fontsize=8)
ax_inset.set_ylabel('Area (1000 km²)', fontsize=10)


# Add values on top of bars (percentage + area)
for bar, (area, percent) in zip(bars, zip(category_area, category_percent)):
    height = bar.get_height()
    if height > 0:
        ax_inset.text(
            bar.get_x() + bar.get_width() / 2, height + 1, 
            f"{area:.1f}k km²\n{percent:.1f}%",  # Show area in thousand km² and percentage
            ha='center', va='bottom', fontsize=8, fontweight='bold'
        )
plt.tight_layout()
plt.savefig('discrete_sens_slope_recharge_trend_with_bar_edges.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import kendalltau
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# Sample dataframe
df = data  # Replace with actual data

# Group data by POINT_X and POINT_Y and calculate the Mann-Kendall trend
def calculate_mann_kendall_trend(group):
    if group['Year'].nunique() > 1:
        tau, p_value = kendalltau(group['Year'], group['recharge'])
        if p_value < 0.05:  # Assuming significance level of 0.05
            return round(tau, 2)  # Round tau to 2 decimal places
        else:
            return 0.0  # Mark as zero trend instead of NaN
    return 0.0  # Not enough data also marked as zero trend

# Calculate the trend for each group
trends_df = df.groupby(['POINT_X', 'POINT_Y']).apply(calculate_mann_kendall_trend).reset_index(name='trend_tau')

# Categorize trends (including zero trend)
def categorize_trend(tau):
    if tau > 0:
        return 'Positive Trend'
    elif tau < 0:
        return 'Negative Trend'
    else:
        return 'Zero Trend'  # Mark zero trend explicitly

trends_df['trend_category'] = trends_df['trend_tau'].apply(categorize_trend)

# Define color palette
custom_palette = {
    'Positive Trend': 'blue',
    'Negative Trend': 'red',
    'Zero Trend': 'grey'
}

# Create main scatter plot
fig, ax = plt.subplots(figsize=(10, 12))

scatter = sns.scatterplot(
    data=trends_df,
    x='POINT_X',
    y='POINT_Y',
    hue='trend_category',
    palette=custom_palette,
    edgecolor=None,
    ax=ax
)

# Add title and labels
ax.set_title('Trend of Upsampled Recharge Change (Mann-Kendall) for Bangladesh (2003-2022)', fontsize=16)
ax.set_xlabel('POINT_X')
ax.set_ylabel('POINT_Y')
ax.legend(title='Trend Category', loc='upper right')

# Count occurrences for bar plot (ensure all categories exist)
category_counts = trends_df['trend_category'].value_counts()

# Ensure all categories exist in the count dictionary
for category in ['Positive Trend', 'Negative Trend', 'Zero Trend']:
    if category not in category_counts:
        category_counts[category] = 0  # Add missing categories with zero count

# Convert counts to area (each point = 4 km²)
total_points = category_counts.sum()
category_area = (category_counts * 4) / 1000  # Convert to thousand km²
category_percent = (category_counts / total_points) * 100  # Calculate percentage

# Create inset bar plot
ax_inset = inset_axes(ax, width="50%", height="20%", loc='upper right')

bars = ax_inset.bar(
    category_counts.index, 
    category_area,  # Use area (in thousand km²) instead of count
    color=[custom_palette[label] for label in category_counts.index], 
    # edgecolor='black',  # Black edges
    # linewidth=1.2
)

ax_inset.set_xticklabels(category_counts.index, 
                         # rotation=45,
                         ha='center', fontsize=8)
ax_inset.set_ylabel('Area (1000 km²)', fontsize=10)

# Add values on top of bars (percentage + area)
for bar, (area, percent) in zip(bars, zip(category_area, category_percent)):
    height = bar.get_height()
    if height > 0:
        ax_inset.text(
            bar.get_x() + bar.get_width() / 2, height + 1, 
            f"{area:.1f}k km²\n{percent:.1f}%",  # Show area in thousand km² and percentage
            ha='center', va='bottom', fontsize=8, fontweight='bold'
        )

plt.tight_layout()
plt.savefig('recharge_trend_mann_kendall_with_percentage_area.png', dpi=300, bbox_inches='tight')
plt.show()


## Visualizing the Trend  of Min GWL

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import theilslopes
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# Sample dataframe
df = data  # Replace with actual data

# Group data by POINT_X and POINT_Y and calculate Sen's Slope
def calculate_sens_slope(group):
    if group['Year'].nunique() > 1:
        slope, intercept, lower, upper = theilslopes(group['Min GWL'], group['Year'])
        return round(slope, 2)  # Round slope to 2 decimal places
    return np.nan  # Not enough data

# Calculate the Sen's Slope for each group and reset index
trends_df = df.groupby(['POINT_X', 'POINT_Y']).apply(calculate_sens_slope).reset_index(name='sens_slope')

# Drop rows with NaN values
trends_df = trends_df.dropna(subset=['sens_slope'])

# Define bins and labels for categorizing slope values
bins = [-1, -0.5, 0, 0.5, 1]
labels = ['-1 to -0.5', '-0.5 to 0', '0 to 0.5', '0.5 to 1']
trends_df['slope_category'] = pd.cut(trends_df['sens_slope'], bins=bins, labels=labels)

# Define custom color palette
custom_palette = {
    '-1 to -0.5': '#8B0000',  # Strong negative trend (dark red)
    '-0.5 to 0': '#FFA07A',   # Moderate negative trend (light salmon)
    '0 to 0.5': '#ADD8E6',    # Moderate positive trend (light blue)
    '0.5 to 1': '#00008B'     # Strong positive trend (deep blue)
}

# Create main scatter plot
fig, ax = plt.subplots(figsize=(10, 12))

scatter = sns.scatterplot(
    data=trends_df,
    x='POINT_X',
    y='POINT_Y',
    hue='slope_category',
    palette=custom_palette,
    edgecolor=None,
    ax=ax
)

# Add title and labels
ax.set_title("Min GWL Trend (Sen's Slope meters/year) for Bangladesh (2003 to 2022)", fontsize=16)
ax.set_xlabel('POINT_X')
ax.set_ylabel('POINT_Y')
ax.legend(title="Slope Category", loc='upper right')

# Count occurrences for bar plot
category_counts = trends_df['slope_category'].value_counts()

# Create inset bar plot
# ax_inset = inset_axes(ax, width="50%", height="23%", loc='upper right')

# Convert counts to area (each point = 4 km²)
total_points = category_counts.sum()
category_area = (category_counts * 4) / 1000  # Convert to thousand km²
category_percent = (category_counts / total_points) * 100  # Calculate percentage

# Create inset bar plot
ax_inset = inset_axes(ax, width="50%", height="20%", loc='upper right')

bars = ax_inset.bar(
    category_counts.index, 
    category_area,  # Use area (in thousand km²) instead of count
    color=[custom_palette[label] for label in category_counts.index], 
    edgecolor='black',  # Black edges
    linewidth=0.5
)

ax_inset.set_xticklabels(category_counts.index, 
                         # rotation=45,
                         ha='center', fontsize=8)
ax_inset.set_ylabel('Area (1000 km²)', fontsize=10)
 # Add values on top of bars (percentage + area)
for bar, (area, percent) in zip(bars, zip(category_area, category_percent)):
    height = bar.get_height()
    if height > 0:
        ax_inset.text(
            bar.get_x() + bar.get_width() / 2, height + 1, 
            f"{area:.1f}k km²\n{percent:.1f}%",  # Show area in thousand km² and percentage
            ha='center', va='bottom', fontsize=8, fontweight='bold'
        )
plt.tight_layout()
plt.savefig('sens_slope_min_gwl_trend_with_bar_edges.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import kendalltau
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# Sample dataframe
df = data  # Replace with actual data

# Function to calculate the Mann-Kendall trend for each (POINT_X, POINT_Y)
def calculate_mann_kendall_trend(group):
    if group['Year'].nunique() > 1:
        tau, p_value = kendalltau(group['Year'], group['Min GWL'])
        return round(tau, 2) if p_value < 0.05 else 0.0  # Use 0.0 for non-significant trends
    return 0.0  # Not enough data

# Apply trend calculation
trends_df = df.groupby(['POINT_X', 'POINT_Y']).apply(calculate_mann_kendall_trend).reset_index(name='trend_tau')

# Categorize trends
def categorize_trend(tau):
    if tau > 0:
        return 'Positive Trend'
    elif tau < 0:
        return 'Negative Trend'
    else:
        return 'Zero Trend'

trends_df['trend_category'] = trends_df['trend_tau'].apply(categorize_trend)

# Define custom colors
custom_palette = {'Positive Trend': 'blue', 'Negative Trend': 'red', 'Zero Trend': 'grey'}

# Create main scatter plot
fig, ax = plt.subplots(figsize=(10, 12))

sns.scatterplot(
    data=trends_df,
    x='POINT_X',
    y='POINT_Y',
    hue='trend_category',
    palette=custom_palette,
    edgecolor=None,
    ax=ax
)

# Add labels
ax.set_title('Trend of Upsampled Min GWL Change (Mann-Kendall) for Bangladesh (2003-2022)', fontsize=16)
ax.set_xlabel('POINT_X')
ax.set_ylabel('POINT_Y')
ax.legend(title='Trend Category', loc='upper right')

# Count occurrences of each trend category
category_counts = trends_df['trend_category'].value_counts()

# Ensure all categories exist
for category in ['Positive Trend', 'Negative Trend', 'Zero Trend']:
    if category not in category_counts:
        category_counts[category] = 0  # Assign zero if missing

# Convert counts to area (each point = 4 km²)
total_points = category_counts.sum()
category_area = (category_counts * 4) / 1000  # Convert to thousand km²
category_percent = (category_counts / total_points) * 100  # Calculate percentage

# Create inset bar plot
ax_inset = inset_axes(ax, width="50%", height="20%", loc='upper right')

bars = ax_inset.bar(
    category_counts.index, 
    category_area, 
    color=[custom_palette[label] for label in category_counts.index], 
    # edgecolor='black', 
    # linewidth=1.2
)

ax_inset.set_xticklabels(category_counts.index,
                         # rotation=45,
                         ha='center', fontsize=8)
ax_inset.set_ylabel('Area (1000 km²)', fontsize=10)

# Add values on top of bars (percentage + area)
for bar, (area, percent) in zip(bars, zip(category_area, category_percent)):
    height = bar.get_height()
    if height > 0:
        ax_inset.text(
            bar.get_x() + bar.get_width() / 2, height + 1, 
            f"{area:.1f}k km²\n{percent:.1f}%",  # Show area in thousand km² and percentage
            ha='center', va='bottom', fontsize=8, fontweight='bold'
        )

plt.tight_layout()
plt.savefig('min_gwl_trend_mann_kendall_with_area.png', dpi=300, bbox_inches='tight')
plt.show()


## Visualizing the Trend  of Max GWL

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import theilslopes
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# Sample dataframe
df = data  # Replace with actual data

# Group data by POINT_X and POINT_Y and calculate Sen's Slope
def calculate_sens_slope(group):
    if group['Year'].nunique() > 1:
        slope, intercept, lower, upper = theilslopes(group['Max GWL'], group['Year'])
        return round(slope, 2)  # Round slope to 2 decimal places
    return np.nan  # Not enough data

# Calculate the Sen's Slope for each group
trends_df = df.groupby(['POINT_X', 'POINT_Y']).apply(calculate_sens_slope).reset_index(name='sens_slope')

# Drop rows with NaN values in sens_slope
trends_df = trends_df.dropna(subset=['sens_slope'])

# Define bins and labels for categorizing slope values
bins = [-1, -0.5, 0, 0.5, 1]
labels = ['-1 to -0.5', '-0.5 to 0', '0 to 0.5', '0.5 to 1']
trends_df['slope_category'] = pd.cut(trends_df['sens_slope'], bins=bins, labels=labels)

# Define custom color palette
custom_palette = {
    '-1 to -0.5': '#8B0000',  # Strong negative trend (dark red)
    '-0.5 to 0': '#FFA07A',   # Moderate negative trend (light salmon)
    '0 to 0.5': '#ADD8E6',    # Moderate positive trend (light blue)
    '0.5 to 1': '#00008B'     # Strong positive trend (deep blue)
}
# Create main scatter plot
fig, ax = plt.subplots(figsize=(10, 12))

scatter = sns.scatterplot(
    data=trends_df,
    x='POINT_X',
    y='POINT_Y',
    hue='slope_category',
    palette=custom_palette,
    edgecolor=None,
    ax=ax
)

# Add title and labels
ax.set_title("Max GWL Trend (Sen's Slope meters/year) for Bangladesh (2003 to 2022)", fontsize=16)
ax.set_xlabel('POINT_X')
ax.set_ylabel('POINT_Y')
ax.legend(title="Slope Category", loc='upper right')

# Count occurrences for bar plot (ensure all categories exist)
category_counts = trends_df['slope_category'].value_counts()

# Ensure all categories exist in the count dictionary
for category in labels:
    if category not in category_counts:
        category_counts[category] = 0  # Add missing categories with zero count

# Convert counts to area in 1000 km² (each point = 4 km²)
category_area = (category_counts * 4) / 1000  # Convert to thousand km²

# Create inset bar plot
ax_inset = inset_axes(ax, width="50%", height="23%", loc='upper right')

bars = ax_inset.bar(
    category_counts.index, 
    category_area, 
    color=[custom_palette[label] for label in category_counts.index], 
    edgecolor='black',  # Black edges
    linewidth=0.1
)
# Convert counts to area (each point = 4 km²)
total_points = category_counts.sum()
category_area = (category_counts * 4) / 1000  # Convert to thousand km²
category_percent = (category_counts / total_points) * 100  # Calculate percentage


ax_inset.set_xticklabels(category_counts.index, ha='center', fontsize=8)
ax_inset.set_ylabel('Area (1000 km²)', fontsize=10)

# Add values on top of bars (percentage + area)
for bar, (area, percent) in zip(bars, zip(category_area, category_percent)):
    height = bar.get_height()
    if height > 0:
        ax_inset.text(
            bar.get_x() + bar.get_width() / 2, height + 1, 
            f"{area:.1f}k km²\n{percent:.1f}%",  # Show area in thousand km² and percentage
            ha='center', va='bottom', fontsize=8, fontweight='bold'
        )
plt.tight_layout()
plt.savefig('discrete_sens_slope_max_gwl_trend_with_area.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import kendalltau
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# Sample dataframe
df = data  # Replace with actual data

# Group data by POINT_X and POINT_Y and calculate the Mann-Kendall trend
def calculate_mann_kendall_trend(group):
    if group['Year'].nunique() > 1:
        tau, p_value = kendalltau(group['Year'], group['Max GWL'])
        return round(tau, 2) if p_value < 0.05 else 0.0  # Use 0.0 for non-significant trends
    return 0.0  # Not enough data also marked as zero trend

# Calculate the trend for each group and reset index
trends_df = df.groupby(['POINT_X', 'POINT_Y']).apply(calculate_mann_kendall_trend).reset_index(name='trend_tau')

# Create a categorical column based on the Mann-Kendall tau value
def categorize_trend(tau):
    if tau > 0:
        return 'Positive Trend'
    elif tau < 0:
        return 'Negative Trend'
    else:
        return 'Zero Trend'

trends_df['trend_category'] = trends_df['trend_tau'].apply(categorize_trend)

# Define the custom color palette for discrete categories
custom_palette = {
    'Positive Trend': 'blue',
    'Negative Trend': 'red',
    'Zero Trend': 'grey'
}

# Create main scatter plot
fig, ax = plt.subplots(figsize=(10, 12))

scatter = sns.scatterplot(
    data=trends_df,
    x='POINT_X',
    y='POINT_Y',
    hue='trend_category',
    palette=custom_palette,
    edgecolor=None,
    ax=ax
)

# Add title and labels
ax.set_title('Trend of Upsampled Max GWL Change (Mann-Kendall) for Bangladesh (2003-2022)', fontsize=16)
ax.set_xlabel('POINT_X')
ax.set_ylabel('POINT_Y')
ax.legend(title='Trend Category', loc='upper right')

# Count occurrences for bar plot (ensure all categories exist)
category_counts = trends_df['trend_category'].value_counts()

# Ensure all categories exist in the count dictionary
for category in ['Positive Trend', 'Negative Trend', 'Zero Trend']:
    if category not in category_counts:
        category_counts[category] = 0  # Add missing categories with zero count

# Convert counts to area (each point = 4 km²)
category_area = (category_counts * 4) / 1000  # Convert to thousand km²

# Create inset bar plot
ax_inset = inset_axes(ax, width="50%", height="23%", loc='upper right')

bars = ax_inset.bar(
    category_counts.index, 
    category_area, 
    color=[custom_palette[label] for label in category_counts.index], 
    # edgecolor='black',  # Black edges
    # linewidth=1.2
)

ax_inset.set_xticklabels(category_counts.index, ha='center', fontsize=8)
ax_inset.set_ylabel('Area (1000 km²)', fontsize=10)

# Add values on top of bars (percentage + area)
for bar, (area, percent) in zip(bars, zip(category_area, category_percent)):
    height = bar.get_height()
    if height > 0:
        ax_inset.text(
            bar.get_x() + bar.get_width() / 2, height + 1, 
            f"{area:.1f}k km²\n{percent:.1f}%",  # Show area in thousand km² and percentage
            ha='center', va='bottom', fontsize=8, fontweight='bold'
        )
plt.tight_layout()
plt.savefig('max_gwl_trend_mann_kendall_with_area.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
!zip SensSlope.zip *sen*
!zip MannKandall.zip *mann_kandall*

In [ ]:
trigger_download('SensSlope.zip')
trigger_download('MannKandall.zip')

In [ ]:
# trigger_download('discrete_sens_slope_recharge_trend.png')

In [ ]:
data[['POINT_X','POINT_Y','Max GWL','Min GWL','recharge','Year','GLDAS_SerialID']].to_csv('Upsampled_2km_resolution_GWL.csv')

In [ ]:
upsampled_gwl=data[['POINT_X','POINT_Y','Max GWL','Min GWL','recharge','Year','GLDAS_SerialID']]

In [ ]:
upsampled_gwl[(upsampled_gwl['Max GWL']>60)]

In [ ]:
!pip install -q rasterio shapely

In [ ]:
import rasterio

# Open the raster file
with rasterio.open("/kaggle/input/water-table-ratio/LOG_WTR_L_01.tif") as src:
    # Print basic metadata
    print("File Metadata:")
    print("----------------")
    print(f"File Name: {src.name}")
    print(f"CRS: {src.crs}")
    print(f"Width, Height: {src.width} x {src.height}")
    print(f"Number of Bands: {src.count}")
    print(f"Data Type: {src.dtypes[0]}")
    print(f"Bounds: {src.bounds}")
    print(f"Resolution: {src.res}")
    
    # Read additional metadata
    print("\nAdditional Metadata:")
    for key, value in src.meta.items():
        print(f"{key}: {value}")


In [ ]:
import rasterio
import pandas as pd
from rasterio.sample import sample_gen
from shapely.geometry import Point

# Load your DataFrame
# Assuming data has columns 'POINT_X' and 'POINT_Y' representing coordinates
# data = pd.read_csv('your_data.csv')  # Uncomment to load data from a file

# Open the recharge raster file
with rasterio.open("/kaggle/input/water-table-ratio/LOG_WTR_L_01.tif") as src:
    # Extract the coordinates from the DataFrame
    coords = [(x, y) for x, y in zip(data['POINT_X'], data['POINT_Y'])]
    
    # Sample the raster at these coordinates
    recharge_values = [val[0] for val in src.sample(coords)]
    
# Add the recharge values as a new column in the DataFrame
data['Recharge_from_'] = recharge_values

# Display the DataFrame with the new 'Recharge' column
print(data.head())


In [ ]:
trigger_download('Upsampled_2km_resolution_GWL.csv')

In [ ]:
 pd.get_dummies(data['lithology']).columns

In [ ]:
importance_df.columns

## Summing the importance of all the lithology for better interpretation

In [ ]:
# Create DataFrame for feature importance
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})

# Filter only for encoded categorical features
encoded_importance_df = importance_df[importance_df['Feature'].isin(X_cat_encoded.columns)].copy()

# Initialize a dictionary to hold summed importances
summed_importance = {}

# Sum importance for features starting with 'lithology'
summed_importance['lithology'] = encoded_importance_df[~encoded_importance_df['Feature'].str.startswith('lithology_MAJORITY')]['Importance'].sum()

# Sum importance for features starting with 'lithology_MAJORITY'
summed_importance['Representative lithology'] = encoded_importance_df[encoded_importance_df['Feature'].str.startswith('lithology_MAJORITY')]['Importance'].sum()

# Convert the results to a DataFrame for easier viewing
summed_importance_df = pd.DataFrame(summed_importance.items(), columns=['Feature', 'Total Importance'])

# Display the summed feature importances
print(summed_importance_df)


In [ ]:
importance_df_2=pd.concat([importance_df[~importance_df['Feature'].isin(X_cat_encoded.columns)],summed_importance_df.rename(columns={'Total Importance':'Importance'})],axis=0)

In [ ]:
importance_df_2.columns

In [ ]:
# summed_importance_df.drop(columns=['lithology_MAJORITY'])

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np

# Assuming importance_df_2 is defined and has the columns 'Feature' and 'Importance'
# Sort the DataFrame by 'Importance' in ascending order
importance_df_2_sorted = importance_df_2.sort_values(by='Importance', ascending=True)

# Calculate the percentage of each importance value
total_importance = importance_df_2_sorted['Importance'].sum()
importance_df_2_sorted['Percentage'] = (importance_df_2_sorted['Importance'] / total_importance) * 100

# Generate a gradient of colors from blue to red
gradient_colors = [mcolors.to_hex(c) for c in plt.cm.coolwarm(np.linspace(0, 1, len(importance_df_2_sorted)))]

# Create a bar plot
plt.figure(figsize=(10, 17))

# Adjust bar positions to increase spacing between ticks
y_positions = np.arange(len(importance_df_2_sorted))  # Original positions
spacing = 1  # Adjust this value for more spacing
y_positions_spaced = y_positions * spacing

bars = plt.barh(
    y_positions_spaced, 
    importance_df_2_sorted['Importance'], 
    color=gradient_colors
)

# Annotate bars with percentages
for i, bar in enumerate(bars):
    plt.text(
        bar.get_width() + 0.02,  # Position to the right of the bar
        bar.get_y() + bar.get_height() / 2,  # Center of the bar
        f"{importance_df_2_sorted['Percentage'].iloc[i]:.1f}%",  # Format percentage to 1 decimal place
        va='center',  # Vertically centered
        fontsize=12,  # Font size for the annotation
        color='black'  # Text color
    )

# Replace y-ticks with spaced positions and corresponding labels
plt.yticks(y_positions_spaced, importance_df_2_sorted['Feature'], fontsize=12)

# Customize axis labels and title
plt.xlabel('Importance', fontsize=14)
plt.ylabel('Features', fontsize=14)
plt.title('Upsampling Model Feature Importance', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Extend x-axis for better visibility
plt.xlim(0, importance_df_2_sorted['Importance'].max() * 1.1)

# Add grid lines
plt.grid(axis='x', linestyle='--', alpha=0.7)

# Adjust layout for better fit
plt.tight_layout()

# Save the figure with DPI 300
plt.savefig('Upsampling_feature_importance.png', dpi=300)

# Display the plot
plt.show()
trigger_download('Upsampling_feature_importance.png')